In [25]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
stopwords_set = set([
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves",
    "you", "your", "yours", "yourself", "yourselves", "he", "him",
    "his", "himself", "she", "her", "hers", "herself", "it", "its",
    "itself", "they", "them", "their", "theirs", "themselves", "what",
    "which", "who", "whom", "this", "that", "these", "those", "am",
    "is", "are", "was", "were", "be", "been", "being", "have", "has",
    "had", "having", "do", "does", "did", "doing", "a", "an", "the",
    "and", "but", "if", "or", "because", "as", "until", "while", "of",
    "at", "by", "for", "with", "about", "against", "between", "into",
    "through", "during", "before", "after", "above", "below", "to",
    "from", "up", "down", "in", "out", "on", "off", "over", "under",
    "again", "further", "then", "once", "here", "there", "when", "where",
    "why", "how", "all", "any", "both", "each", "few", "more", "most",
    "other", "some", "such", "no", "nor", "not", "only", "own", "same",
    "so", "than", "too", "very", "s", "t", "can", "will", "just", "don",
    "should", "now"
])


In [26]:
news_df = pd.read_csv('train.csv')

In [27]:
news_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [28]:
news_df.shape

(20800, 5)

In [29]:
news_df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [30]:
news_df = news_df.fillna(' ')

In [31]:
news_df.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [32]:
news_df['content'] = news_df['author']+" "+news_df['title']

In [33]:
news_df

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [34]:
news_df['content']

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

In [35]:
ps = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if word not in stopwords_set]
    return ' '.join(stemmed_content)


In [36]:
news_df['content'] = news_df['content'].apply(stemming)

In [37]:
news_df['content']

0        darrel lucu hous dem aid didn even see comey l...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

In [38]:
X = news_df['content'].values
y = news_df['label'].values

In [39]:
print(X)


['darrel lucu hous dem aid didn even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [40]:
vector = TfidfVectorizer()
vector.fit(X)
X = vector.transform(X)

In [41]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 211421 stored elements and shape (20800, 17144)>
  Coords	Values
  (0, 267)	0.25894488192633086
  (0, 2485)	0.3524663277134871
  (0, 2961)	0.2366492297246751
  (0, 3603)	0.3450287193908969
  (0, 3795)	0.25935903680643724
  (0, 4011)	0.28443405360208907
  (0, 4978)	0.22367995149083006
  (0, 7012)	0.20970669838645034
  (0, 7700)	0.23761481102346718
  (0, 8638)	0.2800590895926497
  (0, 8918)	0.3485782532641156
  (0, 13483)	0.24599138775354681
  (0, 15697)	0.2730850533643292
  (1, 1499)	0.2939891562094648
  (1, 1896)	0.15521974226349364
  (1, 2225)	0.3827320386859759
  (1, 2815)	0.19094574062359204
  (1, 3571)	0.26373768806048464
  (1, 5508)	0.7143299355715573
  (1, 6823)	0.1904660198296849
  (1, 16813)	0.30071745655510157
  (2, 2945)	0.3179886800654691
  (2, 3105)	0.46097489583229645
  (2, 5394)	0.3866530551182615
  (2, 5973)	0.3474613386728292
  :	:
  (20797, 3646)	0.2115550061362374
  (20797, 7049)	0.21799048897828685
  (2079

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=1)

In [52]:
X_train.shape

(16640, 17144)

In [44]:
X_test.shape

(4160, 17144)

In [54]:
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [55]:
train_y_pred = model.predict(X_train)
print(accuracy_score(train_y_pred,y_train))

0.9868389423076923


In [56]:
input_data = X_test[10]
prediction = model.predict(input_data)

In [57]:
if prediction[0] == 0:
    print('The News Is Real')
else:
    print('The News is Fake')

The News Is Real


In [58]:
news_df['content'][2]

'consortiumnew com truth might get fire'